# DMA on board test

The test procedure is same as "dma_verilog_tb.v", but now written in python instead of verilog, and now it deal with the real DDR

In [ ]:
from pynq import Overlay
overlay = Overlay("base.bit")

from pynq import MMIO
import numpy as np
from pynq import allocate
# print address of axi_dma_0 (customized DMA)
print(hex(overlay.ip_dict['axi_dma_0']['phys_addr']))
# print address of result_checker_0 (customized IP)
print(hex(overlay.ip_dict['result_checker_0']['phys_addr']))

# mape the base address
# 0x60 is the size we want to map, it is enough for both IP
axi_dma = MMIO(hex(overlay.ip_dict['axi_dma_0']['phys_addr']), 0x60)
result_checker = MMIO(hex(overlay.ip_dict['result_checker_0']['phys_addr']), 0x60)

# allocate memory for onboard-test
buffer = allocate(shape=(16, 3742, 2),dtype=np.uint8) # we use 128 x 3742 bits as a frame in test, 2 is for double buffering


In [ ]:
# set buffer address to DMA
# mm2s
# low 32 bits
axi_dma.write(0x10, buffer.physical_address) # write the physical address of the buffer to the DMA
# high 32 bits
axi_dma.write(0x14, (buffer.physical_address)>>32) # write the physical address of the buffer to the DMA
# s2mm
# low 32 bits
axi_dma.write(0x1c, buffer.physical_address) # write the physical address of the buffer to the DMA
# high 32 bits
axi_dma.write(0x20, (buffer.physical_address)>>32) # write the physical address of the buffer to the DMA

In [ ]:
# set test frame num to result_checker
test_frame_num = 1000
result_checker.write(0x20, test_frame_num) # write the number of frames to the result_checker

In [ ]:
import time
# start the test, including error checking and FPS calculation
t1 = time.time()
axi_dma.write(0x00, 0x1) # start the DMA
result_checker.write(0x00, 0x1) # start the result_checker

# wait for the test to finish
while (result_checker.read(0x00) & 0x2) == 0: # it is for done bit, or  we can use 0x4 for idle
    pass

t2 = time.time()

# read the result
error_num = result_checker.read(0x10)
fps = test_frame_num / (t2 - t1)

print("error num: ", error_num)
print("FPS: ", fps)